In [1]:
import torch
import torch.nn as nn
from torch.autograd import Variable
import torch.optim as optim
import torch.nn.functional as F
import numpy as np
import torchvision.transforms as transforms
import torchvision.datasets as vdatasets
import torchvision.utils as vutils
import torchvision.models as vmodels
import torchvision
import random
from PIL import Image
import json
import Augmentor
torch.manual_seed(1)

USE_CUDA = torch.cuda.is_available()

import matplotlib.pyplot as plt
%matplotlib inline

http://cs231n.github.io/transfer-learning/

## 이미지넷 클래스 메타 정보 

In [2]:
idx2cls = json.load(open('imagenet_class.json','r',encoding='utf-8'))

In [3]:
list(idx2cls.items())[:10]

[('260', 'chow, chow chow'),
 ('952', 'fig'),
 ('238', 'Greater Swiss Mountain dog'),
 ('753', 'radiator'),
 ('998', 'ear, spike, capitulum'),
 ('961', 'dough'),
 ('923', 'plate'),
 ('950', 'orange'),
 ('89', 'sulphur-crested cockatoo, Kakatoe galerita, Cacatua galerita'),
 ('76', 'tarantula')]

## Load Pretrained model 

In [17]:
model = vmodels.vgg16(pretrained=True)
model

VGG(
  (features): Sequential(
    (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d (256, 512, kernel_size=(3, 3), 

In [18]:
modules = list(model.children())[0]
feature_extractor = nn.Sequential(*modules)
feature_extractor

Sequential(
  (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (1): ReLU(inplace)
  (2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (3): ReLU(inplace)
  (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (5): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (6): ReLU(inplace)
  (7): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (8): ReLU(inplace)
  (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (10): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (11): ReLU(inplace)
  (12): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (13): ReLU(inplace)
  (14): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (15): ReLU(inplace)
  (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
  (17): Conv2d (256, 512, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
  (18): ReLU(inplace)
  

## 파인튜닝 모델 

프리트레인드 모델을 특징 추출기로 사용

In [34]:
class CNN(nn.Module):
    
    def __init__(self,feature_extractor):
        super(CNN,self).__init__()
        self.feature_extractor = feature_extractor
        # 256*6*6
        self.classifier = nn.Sequential(nn.Linear(512*7*7,512),
                                                    nn.Dropout(0.3),
                                                    nn.ReLU(),
                                                    nn.Linear(512,256),
                                                    nn.Dropout(0.3),
                                                    nn.ReLU(),
                                                    nn.Linear(256,2))
    def forward(self,inputs):
        
        features = self.feature_extractor(inputs)
        features = features.view(-1,512*7*7)
        
        return self.classifier(features)

In [35]:
model = CNN(feature_extractor)
model

CNN(
  (feature_extractor): Sequential(
    (0): Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (1): ReLU(inplace)
    (2): Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (3): ReLU(inplace)
    (4): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (5): Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (6): ReLU(inplace)
    (7): Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (8): ReLU(inplace)
    (9): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (10): Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (11): ReLU(inplace)
    (12): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (13): ReLU(inplace)
    (14): Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
    (15): ReLU(inplace)
    (16): MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
    (17): Conv2d (256, 512, kernel_size

In [36]:
for module in list(model.children())[0]: # feature extractor의 weight는 고정
    print("fix weight", module)
    for param in module.parameters():
        param.requires_grad = False

fix weight Conv2d (3, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
fix weight ReLU(inplace)
fix weight Conv2d (64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
fix weight ReLU(inplace)
fix weight MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
fix weight Conv2d (64, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
fix weight ReLU(inplace)
fix weight Conv2d (128, 128, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
fix weight ReLU(inplace)
fix weight MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
fix weight Conv2d (128, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
fix weight ReLU(inplace)
fix weight Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
fix weight ReLU(inplace)
fix weight Conv2d (256, 256, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1))
fix weight ReLU(inplace)
fix weight MaxPool2d(kernel_size=(2, 2), stride=(2, 2), dilation=(1, 1))
fix weight Conv2d (256, 512, kernel_size=(3, 3), str

## Data pipeline 

In [37]:
normalize = transforms.Normalize(mean=[0.485, 0.456, 0.406],
                                     std=[0.229, 0.224, 0.225])

p = Augmentor.Pipeline()
p.rotate(probability=0.7, max_left_rotation=10, max_right_rotation=10)
p.zoom(probability=0.5, min_factor=1.1, max_factor=1.5)

# validate
preprosessing = transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.ToTensor(),
           normalize,
        ])

# train
train_dataset = vdatasets.ImageFolder(
        "../../data/catdog/train/",
        transforms.Compose([
            transforms.Resize(256),
            transforms.CenterCrop(224),
            transforms.RandomHorizontalFlip(),
            p.torch_transform(),
            transforms.ToTensor(),
            normalize,
        ]))

train_loader = torch.utils.data.DataLoader(dataset=train_dataset,
                                           batch_size=32, 
                                           shuffle=True,
                                           num_workers=2)

In [38]:
print(train_dataset.class_to_idx)
print(len(train_dataset.imgs))

{'dog': 1, 'cat': 0}
2000


## Fine Tuning(Train) 

In [39]:
EPOCH = 10
LR = 0.1
loss_function = nn.CrossEntropyLoss()

# requires_grad가 True인 parameter만 update
optimizer = optim.Adam(filter(lambda p: p.requires_grad, model.parameters()),lr=LR)

In [40]:
model.train()
for epoch in range(EPOCH):
    losses=[]
    for i,(inputs,targets) in enumerate(train_loader):
        inputs, targets = Variable(inputs), Variable(targets)
        model.zero_grad()
        preds = model(inputs)
        loss = loss_function(preds,targets)
        losses.append(loss.data[0])
        loss.backward()
        optimizer.step()
    
    print("[%d/%d] mean_loss : %.3f" % (epoch,EPOCH,np.mean(losses)))

[0/10] mean_loss : 2171.879
[1/10] mean_loss : 93.180
[2/10] mean_loss : 32.878
[3/10] mean_loss : 26.564
[4/10] mean_loss : 18.061
[5/10] mean_loss : 3.452
[6/10] mean_loss : 3.064
[7/10] mean_loss : 32.117
[8/10] mean_loss : 2.823


Process Process-103:
Process Process-104:
Traceback (most recent call last):
Traceback (most recent call last):
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 249, in _bootstrap
    self.run()
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/lib/python3.5/multiprocessing/process.py", line 93, in run
    self._target(*self._args, **self._kwargs)
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/usr/local/lib/python3.5/dist-packages/torch/utils/data/dataloader.py", line 36, in _worker_loop
    r = index_queue.get()
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 342, in get
    with self._rlock:
  File "/usr/lib/python3.5/multiprocessing/queues.py", line 343, in get
    res = self._reader.recv_bytes(

KeyboardInterrupt: 

In [16]:
for idx in pred.topk(5)[1].data.tolist()[0]:
    print(idx2cls[str(idx)])

rock crab, Cancer irroratus
goldfish, Carassius auratus
steam locomotive
leopard, Panthera pardus
beacon, lighthouse, beacon light, pharos
